## This code calculate statistic indicators including AUC, SEN, SPE, PPV, NPV, ACC of ten folds of training, validation and test sets. It also generates each fold's ROC curve with the best threshold.

In [1]:
import time
import re
import os
from plotly import graph_objs as go
from sklearn.metrics import roc_auc_score
from scipy.stats import sem
from sklearn import svm
import pandas as pd
import math
import shutil
import random
from collections import namedtuple
import copy
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.optim.lr_scheduler import _LRScheduler
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from torchvision import utils

from sklearn import decomposition
from sklearn import manifold
from sklearn import metrics
from sklearn import model_selection
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
%matplotlib inline


# import roc

# set seed to make sure the results are reproducible
SEED = 123
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# change directory of data
datapath = os.path.join(".data", "hcb", "cv1")
train_dir = os.path.join(datapath, 'train')
val_dir = os.path.join(datapath, 'val')
test_dir = os.path.join(datapath, 'test')

pretrained_size = (224, 224)
pretrained_means = [0.485, 0.456, 0.406]
pretrained_stds = [0.229, 0.224, 0.225]

train_transforms = transforms.Compose([
    transforms.Resize(pretrained_size),
    #    transforms.ColorJitter(brightness = 1, contrast = 0.5),
    # transforms.RandomHorizontalFlip(0.2),
    # transforms.RandomVerticalFlip(0.2),
    # transforms.RandomCrop(pretrained_size, padding = 10),
    # transforms.CenterCrop(pretrained_size),
    # transforms.ColorJitter(brightness = 1, contrast = 0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=pretrained_means,
                         std=pretrained_stds)
])

test_transforms = transforms.Compose([
    transforms.Resize(pretrained_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=pretrained_means,
                         std=pretrained_stds)
])
# inorder to load img with it's label


class MyImageFolder(datasets.ImageFolder):
    def __getitem__(self, index):
        path, _ = self.imgs[index]  # img path, label
        # return image path
        return super(MyImageFolder, self).__getitem__(index), path


BATCH_SIZE = 64  # 4
# load data from data directory
train_data = MyImageFolder(root=train_dir,
                           transform=train_transforms)
test_data = MyImageFolder(root=test_dir,
                          transform=test_transforms)
valid_data = MyImageFolder(root=val_dir,
                           transform=test_transforms)
train_iterator = data.DataLoader(train_data,
                                 shuffle=True, drop_last=True,
                                 batch_size=BATCH_SIZE)
valid_iterator = data.DataLoader(valid_data, drop_last=True,
                                 batch_size=BATCH_SIZE)
test_iterator = data.DataLoader(test_data, drop_last=True,
                                batch_size=BATCH_SIZE)


# Basic Model

In [2]:
model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet50', pretrained=True)
# change output dimension to what we need
IN_FEATURES = model.fc.in_features
OUTPUT_DIM = 2
model.fc = nn.Linear(IN_FEATURES, OUTPUT_DIM)
fc1 = nn.Linear(IN_FEATURES, 32)
fc2 = nn.Linear(32, OUTPUT_DIM)
model.fc = nn.Sequential(fc1, fc2)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.load_state_dict(torch.load('basic-model-cv1.pt',
                                 map_location=torch.device('cpu')))

# criterion = nn.CrossEntropyLoss()
criterion = nn.CrossEntropyLoss(weight=torch.tensor([1/256, 1/410]).to(device))
criterion = criterion.to(device)

FOUND_LR = 1e-4
weight_decay = 1e-4
params = [
    {'params': model.conv1.parameters(), 'lr': FOUND_LR / 10},
    {'params': model.bn1.parameters(), 'lr': FOUND_LR / 10},
    {'params': model.layer1.parameters(), 'lr': FOUND_LR / 8},
    {'params': model.layer2.parameters(), 'lr': FOUND_LR / 6},
    {'params': model.layer3.parameters(), 'lr': FOUND_LR / 4},
    {'params': model.layer4.parameters(), 'lr': FOUND_LR / 2},
    {'params': model.fc.parameters(), 'lr': FOUND_LR / 2}
]
optimizer = optim.Adam(params, lr=FOUND_LR, weight_decay=weight_decay)
# scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 'min', factor=0.8, patience=5, min_lr=1e-9)


Using cache found in C:\Users\Admin/.cache\torch\hub\pytorch_vision_v0.6.0


In [3]:
# some functions for modelling
def get_predictions(model, iterator):
    model.eval()
    images = []
    labels = []
    probs = []
    top_pred = []
    with torch.no_grad():
        for (xy, _) in iterator:
            x, y = xy
            y_pred = model(x)
            y_prob = F.softmax(y_pred, dim=-1)
            _, _top_pred = torch.max(y_pred, 1)
            images.append(x.cpu())
            labels.append(y.cpu())
            probs.append(y_prob.cpu())
            top_pred.append(_top_pred.cpu())
    images = torch.cat(images, dim=0)
    labels = torch.cat(labels, dim=0)
    probs = torch.cat(probs, dim=0)
    top_pred = torch.cat(top_pred, dim=0)
    return images, labels, probs, top_pred

model.to("cpu")


In [4]:

def roc_results(y_true, y_pred, path):
    r"""
    Return: sensitivity, specificity, ppv, npv, accuracy, AUC, threshold
    this function is used to calculate AUC,sensitivity,specificity,ppv,npv,accuracy
    baesed on each threshold and plot the ROC curve. It will also choose the best threshold, which means that this threshold lead to the highest accuracy.
    input: true, pred, path
    """
# calculate the target vector from roc
    fpr, sen, threshold = metrics.roc_curve(y_true, y_pred)
    spe = 1-fpr
    pos_num = np.sum(y_true == 1)
    neg_num = len(y_true)-pos_num
    tp = sen*pos_num
    tn = spe*neg_num
    fn = pos_num-tp
    fp = neg_num-tn
    ppv = tp/(tp+fp+1e-16)
    npv = tn/(tn+fn+1e-16)
    acc = (tp+tn)/len(y_true)
# the best point data
    j_statistic = sen+spe-1  # (based on J_statistic)
    ind = np.argmax(j_statistic)+0
    pred_p = tp[ind]+fp[ind]
    pred_n = tn[ind]+fn[ind]
    best_acc = acc[ind]
    best_sen = sen[ind]
    best_spe = spe[ind]
    best_ppv = tp[ind]/pred_p
    best_npv = tn[ind]/pred_n
    AUC = metrics.auc(fpr, sen)
# plot
    plt.plot(fpr, sen)
    plt.plot(np.linspace(0, 1, 10), np.linspace(0, 1, 10))
    plt.plot(fpr[ind], sen[ind], 'r.', markersize=9)
    text = "AUC={:.2f}\nSEN={:.2f}\nSPE={:.2f}\nPPV={:.2f}\nNPV={:.2f}\nACC={:.2f}".format(
        AUC, sen[ind], spe[ind], best_ppv, best_npv, best_acc)
    plt.text(0.82, 0.1, text, fontsize=10,
             style="italic", horizontalalignment="center")
    plt.xlabel('1-Specificity')
    plt.ylabel('Sensitivity')
    plt.title('ROC Curve')
    path = path+".jpg"
    plt.savefig(path)
    plt.show()
    return sen, spe, ppv, npv, acc, AUC, threshold

In [5]:
#test cv
for ii in range(1, 11):
    i = str(ii)
    datapath = os.path.join(".data", "hcb", "cv"+i)
    test_dir = os.path.join(datapath, 'test')
    test_data = MyImageFolder(root=test_dir,
                              transform=test_transforms)
    test_iterator = data.DataLoader(test_data, drop_last=True,
                                    batch_size=BATCH_SIZE)
    model.load_state_dict(torch.load(
        'basic-model-cv'+i+'.pt', map_location=torch.device('cpu')))
    cv = i
    path = 'getpred/testcv'+cv
    _, labels, probs, _ = get_predictions(model, test_iterator)
    pred = probs[:, 0]
    sen, spe, ppv, npv, acc, auc, _ = roc_results(
        1-labels.numpy(), pred.numpy(), path)

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same

In [ ]:
#train cv
for ii in range(1, 11):
    i = str(ii)
    # change directory of data
    datapath = os.path.join(".data", "hcb", "cv"+i)
    train_dir = os.path.join(datapath, 'train')
    train_data = MyImageFolder(root=train_dir,
                               transform=train_transforms)
    train_iterator = data.DataLoader(train_data,
                                     shuffle=True, drop_last=True,
                                     batch_size=BATCH_SIZE)
    model.load_state_dict(torch.load(
        'basic-model-cv'+i+'.pt', map_location=torch.device('cpu')))
    cv = i
    path = 'getpred/traincv'+cv
    _, labels, probs, _ = get_predictions(model, train_iterator)
    pred = probs[:, 0]
    sen, spe, ppv, npv, acc, auc, _ = roc_results(
        1-labels.numpy(), pred.numpy(), path)

In [ ]:
#validation cv
for ii in range(1, 11):
    i = str(ii)
    # change directory of data
    datapath = os.path.join(".data", "hcb", "cv"+i)
    val_dir = os.path.join(datapath, 'val')
    valid_data = MyImageFolder(root=val_dir,
                               transform=test_transforms)
    valid_iterator = data.DataLoader(valid_data, drop_last=True,
                                     batch_size=BATCH_SIZE)
    model.load_state_dict(torch.load(
        'basic-model-cv'+i+'.pt', map_location=torch.device('cpu')))
    cv = i
    path = 'getpred/valcv'+cv
    _, labels, probs, _ = get_predictions(model, valid_iterator)
    pred = probs[:, 0]
    sen, spe, ppv, npv, acc, auc, _ = roc_results(
        1-labels.numpy(), pred.numpy(), path)